In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from models.bertopic.utils.bertopic_dataloader import BERTopicDataloader
from models.bertopic.config.embeddings import st_models
from models.bertopic.utils.st_embed_trainer import STEmbeddingTrainer

from sklearn.feature_extraction.text import CountVectorizer

from spacy.lang.el.stop_words import STOP_WORDS as el_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

In [ ]:
loader = BERTopicDataloader('data/data_speeches.csv', 'data/data_statements.csv')
loader.process()

dataset_docs = loader.get_docs()
dataset_sent = loader.get_sentences()

In [ ]:
stopwords = list(el_stop.union(en_stop))
n_grams = (1, 2)

vectorizer_model = CountVectorizer(stop_words=stopwords, 
                                   ngram_range=n_grams)

In [ ]:
st_generator_sentence = STEmbeddingTrainer(st_model_types=st_models, 
                                           dataset=dataset_docs, 
                                           save_path="models/bertopic/data/embeddings/")

st_generator_sentence.train_all()

In [ ]:
# load embeddings from safetensor 
embeddings = st_generator_sentence.get_embeddings()

In [ ]:
topic_model = BERTopic(embedding_model=st_models["gr_media"], 
                      vectorizer_model=vectorizer_model,
                      nr_topics="auto",
                      verbose=True)

In [ ]:
topics, _ = topic_model.fit_transform(dataset_sent)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model = BERTopic(embedding_model=st_greek, 
                       vectorizer_model=vectorizer_model, 
                       nr_topics='auto')
topics, _ = topic_model.fit_transform(docs, embeddings_gr)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model = BERTopic(embedding_model=st_greek, vectorizer_model=vectorizer_model)
topics, _ = topic_model.fit_transform(sentences, embeddings_gr_sentences)